In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

mode = 1 # cas :0, thermometer :1

if mode == 0 :
    # cas
    points = [[8, 30], [22,10], [28,45], [48,22], [68, 7], [70, 40],[90,20]]

    src1 = [113,260]
    src2 = [212,257]
    src3 = [212,298]
    src4 = [115,303]

else:
    # thermometer
    points = [[5,20], [20,10], [19,32], [32,20],[50,10], [50,30], [60,20]]

    src1 = [57,145]
    src2 = [259, 148]
    src3 = [260, 198]
    src4 = [58, 195]

dst1 = [0, 0]
dst2 = [300, 0]
dst3 = [300, 100] 
dst4 = [0, 100]

In [3]:
def show_image(img):
    plt.imshow(img, cmap='gray')
    plt.show()

def crop_image(img, x, y, w, h):
    return img[y:y+h, x:x+w]

def read_image(path):
    img = cv.imread(path)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img

def generate_top_view_image(img, target_width, target_height, 
                            src1, src2, src3, src4, 
                            dst1, dst2, dst3, dst4):
    
    height, width = img.shape[:2] 

    src = np.float32([src1, src2, src3, src4])
    dst = np.float32([dst1, dst2, dst3, dst4])

    matrix = cv.getPerspectiveTransform(src, dst)
    result = cv.warpPerspective(img, matrix, (target_width, target_height))
    
    return result

def draw_points(img, points):
    
    for point in points:
        img[point[0], point[1]] = 100
    return show_image(img)

DIGITS_LOOKUP = {
	(1, 1, 1, 0, 1, 1, 1): 0,
	(0, 0, 1, 0, 0, 1, 0): 1,
	(1, 0, 1, 1, 1, 0, 1): 2,
	(1, 0, 1, 1, 0, 1, 1): 3,
	(0, 1, 1, 1, 0, 1, 0): 4,
	(1, 1, 0, 1, 0, 1, 1): 5,
	(1, 1, 0, 1, 1, 1, 1): 6,
	(1, 0, 1, 0, 0, 1, 0): 7,
	(1, 1, 1, 1, 1, 1, 1): 8,
	(1, 1, 1, 1, 0, 1, 1): 9,
    (0, 0, 0, 0, 0, 0, 0): " "
}

In [6]:

# get images from the folder
path = "./digit_images/"
images = [f for f in os.listdir(path) if f.endswith(".png")]

df = pd.DataFrame(columns=["digit"])

# sort images by number
images.sort(key=lambda x: int(x.split(".")[0]))
print(len(images))

for img in images:
    img = read_image(path + img)
    img = generate_top_view_image(img, 300, 100, src1, src2, src3, src4, dst1, dst2, dst3, dst4)
    
    # seperate digits
    if mode == 0:
        #cas
        digit1 = crop_image(img, 55, 0, 50, 100) 
        digit2 = crop_image(img, 108, 0, 50, 100)
        digit3 = crop_image(img, 170, 0, 50, 100)

    else :
        # thermometer
        digit1 = crop_image(img, 155, 30, 40, 70)
        digit2 = crop_image(img, 190, 30, 40, 70)
        digit3 = crop_image(img, 222, 30, 40, 70)
        
    # make binary image
    digit1 = digit1 < 80
    digit2 = digit2 < 80
    digit3 = digit3 < 80

    digit_indicator1 = [0, 0, 0, 0, 0, 0, 0]
    digit_indicator2 = [0, 0, 0, 0, 0, 0, 0]
    digit_indicator3 = [0, 0, 0, 0, 0, 0, 0]

    # check if points coordinate value is 1
    for i in range(len(points)):
        if digit1[points[i][0], points[i][1]] == 1:
            digit_indicator1[i] = 1
        if digit2[points[i][0], points[i][1]] == 1:
            digit_indicator2[i] = 1
        if digit3[points[i][0], points[i][1]] == 1:
            digit_indicator3[i] = 1
    try:
        # get digit from lookup table
        digit1 = DIGITS_LOOKUP[tuple(digit_indicator1)]
        digit2 = DIGITS_LOOKUP[tuple(digit_indicator2)]
        digit3 = DIGITS_LOOKUP[tuple(digit_indicator3)]

        if mode == 0:
            #cas
            temp = str(digit1) + str(digit2) + "." + str(digit3)
        else:
        # thermometer
            temp = str(digit1) + str(digit2) + str(digit3)
        temp = float(temp)
        print(temp)
        df.loc[len(df)] = [temp]
    
    except:
        print("Error")
        
        print(digit_indicator1)
        print(digit_indicator2)
        print(digit_indicator3)
        show_image(img)
        df.loc[len(df)] = [0]

        continue
# save dataframe to csv file
df.to_csv("data.csv", index=False)

62
32.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
33.0
32.0
32.0
    digit
0    32.0
1    33.0
2    33.0
3    33.0
4    33.0
..    ...
57   33.0
58   33.0
59   33.0
60   32.0
61   32.0

[62 rows x 1 columns]


In [7]:
# get temp every n seconds from csvs

time_gap = 1 # n
# read csv file
df = pd.read_csv('./data.csv')
# get every n seconds
df = df.iloc[::time_gap, :]
# save
df.to_csv('./digit_' + str(time_gap) + '.csv', index=False)